# Reading data from datasets

In [1]:
import util
import numpy as np
import random

In [2]:
# Available datasets: p01 to p08, c08 to c11
[[capacity], weights, profits, optimal_solution] = util.read_data('c10')
optimal_profits = np.multiply(optimal_solution, profits).sum()

In [3]:
capacity

863123059

In [4]:
weights

[18427422,
 37073025,
 37645807,
 48092435,
 19026777,
 31341879,
 19087179,
 17172221,
 27936937,
 39331629,
 15437562,
 43701941,
 45763844,
 26377355,
 24377559,
 12835054,
 36350045,
 44765946,
 7367789,
 13814390,
 28726138,
 42582063,
 38119242,
 20600442,
 39565042,
 28170306,
 41075409,
 18902330,
 40210230,
 44940994,
 15056905,
 28959901,
 12360624,
 13078452,
 43112045,
 40850230,
 15187270,
 10146034,
 37232217,
 28614856,
 42950615,
 22852553,
 42192942,
 32288911,
 5594339,
 32161935,
 14809690,
 25998715,
 29317555,
 22586816,
 18676667,
 18108625,
 46588883,
 41973770,
 38906250,
 10766129,
 9411164,
 43438691,
 40164062,
 34040281]

In [5]:
profits

[169488031,
 140262926,
 483734912,
 491097912,
 303416095,
 477302881,
 436649602,
 301362139,
 133935212,
 221594758,
 285644467,
 456356356,
 489302486,
 330014738,
 491167636,
 278911505,
 246558412,
 298681929,
 399531626,
 428923627,
 336739401,
 220736391,
 432335041,
 234327908,
 309593963,
 175556420,
 194178727,
 160409810,
 65651368,
 202784888,
 204529693,
 406762738,
 261989333,
 400251899,
 352098674,
 328003137,
 425021683,
 53544692,
 158420987,
 281420743,
 474052893,
 105973222,
 485972546,
 129075676,
 477194324,
 386877797,
 349982424,
 305984825,
 406748303,
 249876999,
 161794564,
 203060787,
 326714191,
 380247766,
 466922220,
 420421280,
 290201407,
 252978376,
 385829634,
 450539762]

In [6]:
optimal_solution

[0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 0,
 1,
 1,
 1,
 0,
 0,
 1]

In [7]:
optimal_profits

13323209245

# Actual Computation

In [8]:
def generate_candidates(solution_length, num_of_candidates):
    return [util.join_to_str([1 if random.uniform(0, 1) >= 0.5 else 0 for i in range(solution_length)]) for j in range(num_of_candidates)]

In [9]:
def get_profits(s, c, w, p):
    s_num = util.split_to_num(s)
    total_weights = np.multiply(s_num, w).sum()
    if total_weights <= c:
        return np.multiply(s_num, p).sum()
    else:
        return 0

In [10]:
def get_winner(item_1, item_2, c, w, p):
    return item_1 if get_profits(item_1, c, w, p) > get_profits(item_2, c, w, p) else item_2

In [11]:
def get_child(item_1, item_2):
    indices = random.sample(range(len(item_1)+1), 2)
    indices.sort()
    [start_index, end_index] = indices
    return item_1[0:start_index] + item_2[start_index:end_index] + item_1[end_index:]

In [12]:
def mutate_word(word, chance):
    new_word = ''
    for char in word:
        if (random.uniform(0, 1) < chance):
            new_word += '0' if char is '1' else '1'
        else:
            new_word += char
    return new_word

In [13]:
def knapsack_genetic(c, w, p, generations):
    generation_count = 0
    best_answer_so_far = ''
    best_answer_score_so_far = 0
    
    # number of generations
    while generation_count < generations:
        generation_count += 1
        
        # create first generation with 10 candidates
        while True:
            candidates = generate_candidates(len(w), 10)
            scores = [get_profits(candidate, c, w, p) for candidate in candidates]
            # to get at least 1 non-zero profit candidate
            if (max(scores) > 0):
                break

        # retain the highest value item
        max_item_score = max(scores)
        max_item_index = scores.index(max_item_score)
        max_item = candidates[max_item_index]
        next_gen_candidates = []
        next_gen_candidates.append(max_item)

        # tournament
        while len(next_gen_candidates) < len(candidates):
            [item_1, item_2] = random.sample(candidates, 2)
            [item_3, item_4] = random.sample(candidates, 2)
            winner_1 = get_winner(item_1, item_2, c, w, p)
            winner_2 = get_winner(item_3, item_4, c, w, p)
            child = get_child(winner_1, winner_2)
            next_gen_candidates.append(child)

        # mutation - with chance of 1%
        mutated_next_gen_candidates = [mutate_word(candidate, 0.01) for candidate in next_gen_candidates]

        # determine best score
        mutated_next_gen_scores = [get_profits(candidate, c, w, p) for candidate in mutated_next_gen_candidates]
        max_item_score = max(scores)
        max_item_index = scores.index(max_item_score)
        max_item = candidates[max_item_index]
        if (max_item_score > best_answer_score_so_far):
            best_answer_so_far = max_item
            best_answer_score_so_far = max_item_score

    return best_answer_score_so_far, best_answer_so_far

In [14]:
computed_profits, s = knapsack_genetic(capacity, weights, profits, 1000000)
computed_solution = util.split_to_num(s)

In [15]:
print(f'optiomal profits: {optimal_profits}')
print(f'computed_profits: {computed_profits}')

optiomal profits: 13323209245
computed_profits: 12084159837


In [16]:
print(f'optimal_solution: \n{optimal_solution}')
print(f'computed_solution: \n{computed_solution}')

optimal_solution: 
[0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1]
computed_solution: 
[1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1]
